# Waffle House Formal Authentication Tokenizer Concept


In [1]:
# Import libraries

import pandas as pd
from huggingface_hub import notebook_login
from pprint import pprint
from transformers import AutoTokenizer

In [2]:
# Login to hugging face
notebook_login()

In [8]:
# Import tokenizer

tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

# Waffle House Formal Authentication

In [9]:
# Define Waffle House markers and expected results (expanded with all relevant tokens)
waffle_tokens = {
    "jelly_bottom": "scrambled_eggs",
    "hashbrowns_top": "hashbrowns_selected",
    "tomato_top": "tomato_selected",
    "napkin_top": "oatmeal_selected",
    "jelly_flipped": "wheat_toast",
    "apple_jelly_bottom": "raisin_toast",
    "extra_jelly": "extra_toast",
    "butter_underneath": "dry_toast",
    "mustard_under_jelly": "three_scrambled_eggs",
    "two_jelly_stack": "four_scrambled_eggs"
}

# Tokenize each marker phrase
tokenized_markers = {k: tokenizer(v)["input_ids"] for k, v in waffle_tokens.items()}


from IPython.display import display
display(tokenized_markers)


{'jelly_bottom': [101, 13988, 168, 6471, 102],
 'hashbrowns_top': [101, 1144, 1324, 12725, 6540, 1116, 168, 2700, 102],
 'tomato_top': [101, 26422, 168, 2700, 102],
 'napkin_top': [101, 184, 2980, 3263, 1348, 168, 2700, 102],
 'jelly_flipped': [101, 11773, 168, 17458, 102],
 'apple_jelly_bottom': [101, 187, 15837, 1394, 168, 17458, 102],
 'extra_jelly': [101, 3908, 168, 17458, 102],
 'butter_underneath': [101, 3712, 168, 17458, 102],
 'mustard_under_jelly': [101, 1210, 168, 13988, 168, 6471, 102],
 'two_jelly_stack': [101, 1300, 168, 13988, 168, 6471, 102]}

In [10]:
# Function to authenticate tokens
def authenticate_user(tokens):
    for token, expected_token in tokenized_markers.items():
        if tokens.get(token) != expected_token:
            return False
    return True

In [11]:
# Sample user tokens to authenticate
user_tokens = {
    "jelly_bottom": tokenizer("scrambled_eggs")["input_ids"],
    "hashbrowns_top": tokenizer("hashbrowns_selected")["input_ids"]
}
auth_status = authenticate_user(user_tokens)
print("Access Granted" if auth_status else "Access Denied")


Access Denied


### Verification Functions Based on SMV Rules

In [12]:
# Rule validation functions
def check_scrambled_eggs_rule(tokens):
    return tokens.get("jelly_bottom") == tokenized_markers["jelly_bottom"]

def check_grits_rule(tokens):
    return (tokens.get("hashbrowns_top") == tokenized_markers["hashbrowns_top"] and "grits" not in tokens)

def check_oatmeal_rule(tokens):
    return tokens.get("napkin_top") == tokenized_markers["napkin_top"]

def check_conflict_rule(tokens):
    return not (tokens.get("hashbrowns_top") and tokens.get("tomato_top"))


### Order Validation Function

In [13]:
def validate_order(tokens):
    
    rules = [
        check_scrambled_eggs_rule(tokens),
        check_grits_rule(tokens),
        check_oatmeal_rule(tokens),
        check_conflict_rule(tokens)
    ]
    return all(rules)



In [14]:
# Validate sample order
if validate_order(user_tokens):
    print("Order Valid - Configuration matches SMV rules")
else:
    print("Order Invalid - Conflict or Rule Violation Detected")

Order Invalid - Conflict or Rule Violation Detected


### Export to SMV Format for nuXmv Verification

In [103]:
# Export function to create SMV file
def export_to_smv(tokens, filename="generated_wafflehouse.smv"):
    with open(filename, 'w') as smv_file:
        smv_file.write("MODULE main\nVAR\n")

        # List of all relevant variables
        all_vars = list(waffle_tokens.keys()) + ["scrambled_eggs", "grits", "oatmeal", "brown_sugar_top"]
        for var in all_vars:
            smv_file.write(f"    {var} : boolean;\n")

        # INIT section: all variables false
        smv_file.write("\nINIT\n    ")
        init_conditions = " & ".join([f"!{var}" for var in all_vars])
        smv_file.write(init_conditions + ";\n\n")

        # TRANS section based on rules
        smv_file.write("TRANS\n")
        trans_conditions = [
            "(jelly_bottom -> scrambled_eggs)",
            "(hashbrowns_top -> (scrambled_eggs & !grits))",
            "(oatmeal -> (napkin_top & brown_sugar_top))"
        ]
        smv_file.write("    " + " &\n    ".join(trans_conditions) + ";\n\n")

        # LTL specifications
        smv_file.write("-- Formal Specifications\n")
        smv_file.write("LTLSPEC G (jelly_bottom -> scrambled_eggs);\n")
        smv_file.write("LTLSPEC G (oatmeal -> F(napkin_top & brown_sugar_top));\n")
        smv_file.write("LTLSPEC !(hashbrowns_top & tomato_top);\n")

    print(f"SMV model exported to {filename}")

In [104]:
# Export the tokens for formal verification
export_to_smv(user_tokens)

In [ ]:
# Export function to create SMV file for LLM response validation
def export_llm_response_to_smv(response_text, filename="llm_response_validation.smv"):
    with open(filename, 'w') as smv_file:
        smv_file.write("MODULE main\nVAR\n")

        # Declare variables representing key elements in the response
        smv_file.write("    acknowledged_order : boolean;\n")
        smv_file.write("    included_items : boolean;\n")
        smv_file.write("    explained_plate_markings : boolean;\n")

        # Define initial conditions (assuming all start as FALSE)
        smv_file.write("\nINIT\n")
        smv_file.write("    !acknowledged_order & !included_items & !explained_plate_markings;\n")

        # TRANS section: Define transitions based on the response content
        smv_file.write("\nTRANS\n")
        smv_file.write("    next(acknowledged_order) = response_acknowledges_order;\n")
        smv_file.write("    next(included_items) = response_includes_items;\n")
        smv_file.write("    next(explained_plate_markings) = response_explains_plate_markings;\n")

        # Define properties to check (LTL specifications)
        smv_file.write("\n-- Specifications\n")
        smv_file.write("LTLSPEC G (acknowledged_order & included_items & explained_plate_markings);\n")

        # Define DEFINE section to set the conditions based on the response_text
        smv_file.write("\nDEFINE\n")
        smv_file.write(f"    response_acknowledges_order := {str('thank you' in response_text.lower() or 'sure' in response_text.lower()).upper()};\n")
        smv_file.write(f"    response_includes_items := {str(any(item in response_text.lower() for item in waffle_tokens.keys())).upper()};\n")
        smv_file.write(f"    response_explains_plate_markings := {str(any(marker in response_text.lower() for marker in waffle_tokens.values())).upper()};\n")

    print(f"SMV model for LLM response validation exported to {filename}")
